# Problem 1
## a. Using variable elimination, calculate the probability that a student who did well on the exam understands the material.P(+u|+e). Show your work.

## b. Given the Bayesian network, are T and U independent? Why

P(+e) = 

# Problem 2

In [12]:
import numpy as np
import pandas as pd
from IPython.display import display, Markdown
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import cross_val_score
import statistics
data = pd.read_csv("Life Expectancy Data.csv")
data.describe()


,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
count,2938.000000,2928.000000,2928.000000,2938.000000,2744.000000,2938.000000,2385.000000,2938.000000,2904.000000,2938.000000,2919.000000,2712.00000,2919.000000,2938.000000,2490.000000,2.286000e+03,2904.000000,2904.000000,2771.000000,2775.000000
mean,2007.518720,69.224932,164.796448,30.303948,4.602861,738.251295,80.940461,2419.592240,38.321247,42.035739,82.550188,5.93819,82.324084,1.742103,7483.158469,1.275338e+07,4.839704,4.870317,0.627551,11.992793
std,4.613841,9.523867,124.292079,117.926501,4.052413,1987.914858,25.070016,11467.272489,20.044034,160.445548,23.428046,2.49832,23.716912,5.077785,14270.169342,6.101210e+07,4.420195,4.508882,0.210904,3.358920
min,2000.000000,36.300000,1.000000,0.000000,0.010000,0.000000,1.000000,0.000000,1.000000,0.000000,3.000000,0.37000,2.000000,0.100000,1.681350,3.400000e+01,0.100000,0.100000,0.000000,0.000000
25%,2004.000000,63.100000,74.000000,0.000000,0.877500,4.685343,77.000000,0.000000,19.300000,0.000000,78.000000,4.26000,78.000000,0.100000,463.935626,1.957932e+05,1.600000,1.500000,0.493000,10.100000
50%,2008.000000,72.100000,144.000000,3.000000,3.755000,64.912906,92.000000,17.000000,43.500000,4.000000,93.000000,5.75500,93.000000,0.100000,1766.947595,1.386542e+06,3.300000,3.300000,0.677000,12.300000
75%,2012.000000,75.700000,228.000000,22.000000,7.702500,441.534144,97.000000,360.250000,56.200000,28.000000,97.000000,7.49250,97.000000,0.800000,5910.806335,7.420359e+06,7.200000,7.200000,0.779000,14.300000
max,2015.000000,89.000000,723.000000,1800.000000,17.870000,19479.911610,99.000000,212183.000000,87.300000,2500.000000,99.000000,17.60000,99.000000,50.600000,119172.741800,1.293859e+09,27.700000,28.600000,0.948000,20.700000


# Divide the data into a training set and a testing set
I am dividing the training set and testing set by selecting 25% of the data for training (with replacement) and 75% as the test data set.


In [13]:
train = data.sample(frac=0.25)
test = data.sample(frac=0.75)

# Using your training set, determine which variables are actually affecting the life expectancy.How did you arrive at that conclusion?

In [14]:
# To find correlation of each variable perform linear regression in relation to each variable
def print_col_correlations(data):
    filtered = data.values[:,:]
    filtered=filtered[:,4:]
    i = 4
    r_squared_data = []
    for col in filtered.T:
        col = col.astype(float)
        col = np.array(col)

        y =  data.values[:,3]

        y = y.astype(float)
        nans = np.isnan(y)
        y = y[~nans]

        col = col[~nans]

        col_nans = np.isnan(col)
        col=col[~col_nans]
        y = y[~col_nans]
        reg = LinearRegression().fit(col.reshape(-1,1), y)
        #print("score for: "+str(data.columns[i]))
        score = reg.score(col.reshape(-1,1), y)
        r_squared_data.append([data.columns[i],score])
        i+=1
    return pd.DataFrame(
        r_squared_data,columns=["measure_name","r squared"])
print_col_correlations(train)


,measure_name,r squared
0,Adult Mortality,0.428285
1,infant deaths,0.046285
2,Alcohol,0.117134
3,percentage expenditure,0.153567
4,Hepatitis B,0.092535
5,Measles,0.027143
6,BMI,0.325734
7,under-five deaths,0.057413
8,Polio,0.221880
9,Total expenditure,0.022186


Infant mortality ,Hepatitis B, Measles, under-five deaths, Total expenditure and Population	have low correlation with life expectancy. Therefore they will be removed from the analysis.

In [15]:
def drop_rows(data):
    data = data.dropna()
    data.drop('infant deaths',1)
    data.drop('Hepatitis B',1)
    data.drop('Measles ',1)
    data.drop('under-five deaths ',1)
    data.drop('Total expenditure',1)
    data.drop('Population',1)
    return data
def make_model(data,normalize=False):
    
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = LinearRegression(normalize=normalize).fit(data.values[:,4:], life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg

make_model(train)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

# Evaluate how well does your model predict life expectancy


In [16]:
def eval_model(model,train):

    # dropping rows from test dataset
    test_cleaned = drop_rows(test)
    x = test_cleaned.values[:,4:]
    y=test_cleaned.values[:,3]
    #for i in model.coef_:
    #    print(i)
    #for i in test_cleaned.axes[0]:
    #    print(i)
    data = []
    try:
        for coef,axes in zip(model.coef_,test_cleaned.axes[1]):
            data.append([axes,coef])
        frame = pd.DataFrame(data,columns=["Axis Name","Coefficient"])
        display(frame)
    except:
        a=0

    countries = {}
    for row in test_cleaned.values:
        #print(row[0])
        #print(row[4:])
        prediction = model.predict(row[4:].reshape(1, -1))[0]
        actual = row[3]
        err = prediction-actual
        #print(repr(err))
        try:
            countries[row[0]].append(err)
        except:
            countries[row[0]]=[err]
    #print(countries)
    country_stdev_arr = []
    for country in countries:
        stdev=0
        if(len(countries[country])>1):
            stdev = statistics.stdev(countries[country])
        #print(stdev)
        country_stdev_arr.append([country,stdev])
    display(pd.DataFrame(country_stdev_arr,columns=["Country","Standard Deviation"]))
model = make_model(train)
eval_model(model,train)

,Axis Name,Coefficient
0,Country,-1.673030e-02
1,Year,7.003552e-02
2,Status,-1.806005e-01
3,Life expectancy,8.779985e-04
4,Adult Mortality,-5.855296e-04
5,infant deaths,-1.095078e-05
6,Alcohol,3.321609e-02
7,percentage expenditure,-5.119188e-02
8,Hepatitis B,5.967161e-03
9,Measles,5.662884e-02


,Country,Standard Deviation
0,Indonesia,1.391569
1,Haiti,0.207971
2,Paraguay,0.959220
3,Swaziland,6.117157
4,Ireland,0.612098
...,...,...
127,Croatia,1.092439
128,Montenegro,1.036056
129,Poland,0.941527
130,Sweden,4.376688


The results show the standard deviation for the difference between real life expectancy and real life expectancy based on country.

# Scikit-learn offers two other types of regression, Ridge and Lasso, whichhelp with reducing the magnitude of the coefficientsand reduces overfitting. Using regularization,determine if your model improves using the Ridge or Lasso regression. See which alphavalues provide the best results. Describe your results

In [17]:
def make_ridge(data,normalize=False,alpha=1.0):
    
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = Ridge(normalize=normalize,alpha=alpha).fit(data.values[:,4:], life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg,score
ridge,score = make_ridge(train)
eval_model(ridge,train)

/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=4.25378e-18): result may not be accurate.
  overwrite_a=True).T


,Axis Name,Coefficient
0,Country,-1.683343e-02
1,Year,7.045294e-02
2,Status,-1.751630e-01
3,Life expectancy,8.700601e-04
4,Adult Mortality,-4.687581e-04
5,infant deaths,-1.068622e-05
6,Alcohol,3.407182e-02
7,percentage expenditure,-5.146457e-02
8,Hepatitis B,5.836993e-03
9,Measles,5.847566e-02


,Country,Standard Deviation
0,Indonesia,1.397760
1,Haiti,0.172019
2,Paraguay,0.969790
3,Swaziland,6.149613
4,Ireland,0.661280
...,...,...
127,Croatia,1.116600
128,Montenegro,1.026311
129,Poland,0.947445
130,Sweden,4.412484


In [18]:
def make_lasso(data,normalize=False,alpha=1.0):
    data = drop_rows(data)
    life_expect = data.values[:,3]
    #print(life_expect)
    reg = Lasso(normalize=normalize,alpha=alpha).fit(data.values[:,4:],
        life_expect)
    score = reg.score(data.values[:,4:], life_expect)
    return reg,score
lasso,score = make_lasso(train)
eval_model(lasso,train)

/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3504.1158005044695, tolerance: 2.91604826869159
  positive)


,Axis Name,Coefficient
0,Country,-1.947819e-02
1,Year,6.690594e-02
2,Status,-0.000000e+00
3,Life expectancy,7.077198e-04
4,Adult Mortality,6.551964e-04
5,infant deaths,-5.853898e-06
6,Alcohol,5.639894e-02
7,percentage expenditure,-5.014146e-02
8,Hepatitis B,5.657736e-03
9,Measles,0.000000e+00


,Country,Standard Deviation
0,Indonesia,1.282451
1,Haiti,0.638881
2,Paraguay,1.002423
3,Swaziland,6.033462
4,Ireland,1.659911
...,...,...
127,Croatia,1.696678
128,Montenegro,0.686548
129,Poland,1.086741
130,Sweden,4.980480


## Tuning alpha

In [19]:
max_lasso = 0
max_lasso_a=0
best_lasso=""
max_ridge = 0
max_ridge_a=0
best_ridge=""
for a_int in range(1,100,1):
    a= (float (a_int))/10.0
    lasso_model,lasso_score=make_lasso(train,alpha=a)
    if(lasso_score>max_lasso):
        max_lasso=lasso_score
        max_lasso_a=a
        best_lasso=lasso_model
    ridge_model,ridge_score=make_ridge(train,alpha=a)
    if(ridge_score>max_ridge):
        max_ridge=ridge_score
        max_ridge_a=a
        best_ridge=ridge_model
display(Markdown("### Max Lasso Alpha: "+str(max_lasso_a)))
display(Markdown("### Max Ridge Alpha: "+str(max_ridge_a)))

y not be accurate.
  overwrite_a=True).T
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 131.19696419176307, tolerance: 2.91604826869159
  positive)
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=5.89044e-18): result may not be accurate.
  overwrite_a=True).T
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 122.1293767129373, tolerance: 2.91604826869159
  positive)
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_ridge.py:148: LinAlgWarning: Ill-conditioned matrix (rcond=5.94688e-18): result may not be accurate.
  overwrite_a=True).T
/home/nick/.local/lib/python3.7/site-pac

### Max Lasso Alpha: 0.1

### Max Ridge Alpha: 0.1

In [21]:
def eval_cross_validate(model,train,model_name:str):
    # dropping rows from test dataset
    test_cleaned = drop_rows(train)
    x = test_cleaned.values[:,4:]
    y=test_cleaned.values[:,3]
    #print(x)
    #print(y)
    scores = cross_val_score(model,x,y,cv=5)
    display(Markdown("## Mean "+model_name+ " Score: "+format(".2f",scores.mean())))
eval_cross_validate(best_lasso,train,"Lasso")
eval_cross_validate(best_ridge,train,"Ridge")

/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2270.1143071377924, tolerance: 2.2976698128654967
  positive)
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2216.4302538253924, tolerance: 2.276370374269006
  positive)
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2100.8735050413607, tolerance: 2.2964878947368415
  positive)
/home/nick/.local/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

TypeError: format() argument 2 must be str, not numpy.float64